In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax, MaxPooling2D,BatchNormalization, ELU
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
from keras import regularizers

In [2]:
tf.__version__

'2.13.0'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Pre-Processing

Open **dan_train.csv** file and split the games into a list.
Every row of csv: `DL0000000001,B,B[pd],W[dp],B[pp],W[dc],B[de],...`.

Columns are:

    1. DL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [4]:
df = open('/content/drive/Shareddrives/AICUP圍棋Dataset/dan_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [5]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0

Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [6]:
def prepare_input(moves):
    x = np.zeros((19, 19, 6))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [7]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 100160, Total Moves: 22853380


The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

In [8]:
import random
x = []
y = []
games = random.sample(games, 500)
for game in games:
    moves_list = game.split(',')
    for count, move in enumerate(moves_list):
        x.append(prepare_input(moves_list[:count]))
        y.append(prepare_label(moves_list[count]))
x = np.array(x)
y = np.array(y)

In [9]:
x.shape

(113370, 19, 19, 6)

In [10]:
y.shape

(113370,)

In [11]:
y_one_hot = tf.one_hot(y, depth=19*19)

Dataset splitting: 90% Training, 10% validation

In [12]:
x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

# Training

### Simple DCNN Model:

In [13]:
def create_model():
    inputs = Input(shape=(19, 19, 6))
    outputs = Conv2D(kernel_size=16, filters=4, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=16, filters=4, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=8, filters=2, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=4, filters=2, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=4, filters=1, padding='same', activation='relu',kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)

    outputs = ELU()(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    model = Model(inputs, outputs)

    opt = Adam(learning_rate=0.005)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [14]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 19, 6)]       0         
                                                                 
 conv2d (Conv2D)             (None, 19, 19, 4)         6148      
                                                                 
 batch_normalization (Batch  (None, 19, 19, 4)         16        
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 4)         4100      
                                                                 
 batch_normalization_1 (Bat  (None, 19, 19, 4)         16        
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 19, 2)         514   

In [15]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

In [16]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 256,
    epochs = 40,
    validation_data=(x_val, y_val),
)

Epoch 1/40
399/399 [==============================] - 20s 15ms/step - loss: 5.4797 - accuracy: 0.0175 - val_loss: 5.2760 - val_accuracy: 0.0260
Epoch 2/40
399/399 [==============================] - 5s 12ms/step - loss: 4.7729 - accuracy: 0.0520 - val_loss: 4.8389 - val_accuracy: 0.0546
Epoch 3/40
399/399 [==============================] - 5s 14ms/step - loss: 4.6975 - accuracy: 0.0635 - val_loss: 4.8899 - val_accuracy: 0.0561
Epoch 4/40
399/399 [==============================] - 5s 12ms/step - loss: 4.6653 - accuracy: 0.0666 - val_loss: 4.7034 - val_accuracy: 0.0602
Epoch 5/40
399/399 [==============================] - 5s 12ms/step - loss: 4.6590 - accuracy: 0.0685 - val_loss: 4.6409 - val_accuracy: 0.0722
Epoch 6/40
399/399 [==============================] - 5s 13ms/step - loss: 4.6471 - accuracy: 0.0706 - val_loss: 4.6980 - val_accuracy: 0.0663
Epoch 7/40
399/399 [==============================] - 5s 14ms/step - loss: 4.6399 - accuracy: 0.0699 - val_loss: 4.7459 - val_accuracy: 0.057

In [17]:
model.save('/content/drive/Shareddrives/AICUP圍棋Dataset/model_dan_tutorial.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Testing

**PublicUpload.csv** must be in the following form:
```
DTPU0000000001,id,qr,pq,pd,ab
DTPU0000000002,ao,ab,ha,ff,qd
DTPU0000000003,qd,gd,fh,ed,fa
DTPU0000000004,pr,ba,dq,hh,aj
DTPU0000000005,ph,jh,af,df,gj
```

- Column 1: Game ID
- Column 2: Predicted Moves, up to 5 predictions for each game

The code block below is to use **dan_test_public.csv** to predict and save the results in required form. It generates the best 5 predictions for each sample and convert them to character coordinates.

In [18]:
def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

In [19]:
df = open('/content/drive/Shareddrives/AICUP圍棋Dataset/dan_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to PublicUpload.csv
with open('/content/drive/Shareddrives/AICUP圍棋Dataset/PublicUpload.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

344/344 [==============================] - 1s 2ms/step


# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!

In [20]:
print(prediction_chars)

[['ol' 'nm' 'mn' 'mk' 'll']
 ['jn' 'im' 'jl' 'km' 'kn']
 ['if' 'je' 'kd' 'id' 'jc']
 ...
 ['fp' 'gq' 'eq' 'go' 'hp']
 ['gf' 'hg' 'fg' 'he' 'gh']
 ['il' 'gk' 'jl' 'im' 'hm']]
